In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = '/content/drive/MyDrive/cs231n/2022/assignment1'
assert FOLDERNAME is not None, '/content/drive/MyDrive/cs231n/2022/assignment1'

In [ ]:
# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/MyDrive/cs231n/2022/assignment1')

In [ ]:
# This downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd '/content/drive/MyDrive/cs231n/2022/assignment1/cs231n/datasets'
!bash get_datasets.sh
%cd '/content/drive/MyDrive/cs231n/2022/assignment1'

# k-Nearest Neighbor (kNN) exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

The kNN classifier consists of two stages:

- During training, the classifier takes the training data and simply remembers it
  - 훈련 중, classifier는 training data를 가지고 simply remember 한다.
- During testing, kNN classifies every test image by comparing to all training images and transfering the labels of the k most similar training examples
  - testing 중, kNN은 모든 test image를 classify한다. training images 모두와 비교하여 label을 가장 training example과 가까운 곳으로 옮겨서.
- The value of k is cross-validated
  - k의 값은 cross-validated

In this exercise you will implement these steps and understand the basic Image Classification pipeline, cross-validation, and gain proficiency in writing efficient, vectorized code.
- image classification pipeline 이해
- cross-validation 이해
- 효율적이고 vectorized code 작성 능력 습득

In [ ]:
# Run some setup code for this notebook.

import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# 데이터 불러오기

데이터를 불러오고, 데이터의 shape을 파악한다.
1. 데이터 불러오기 : load_CIFAR10('data주소')
2. 데이터 shape 파악 : .shape 사용
- Training : 50000개, test 10000개
- cifar10 : 32 32 3 의 shape

In [ ]:
# Load the raw CIFAR-10 data.
cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'

# Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
try:
   del X_train, y_train
   del X_test, y_test
   print('Clear previously loaded data.')
except:
   pass

X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

# As a sanity check, we print out the size of the training and test data.
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

# Data visualization

flatnonzero(a): a에서 0이 아닌 index를 반환한다.  
random.choice(a) : a에서 랜덤으로 하나 뽑아주기.  
plt.subplot(row, column, index) : 여러 개를 하나의 화면에 띄우기  


In [ ]:
# Visualize some examples from the dataset.
# We show a few examples of training images from each class.
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    #train data의 class 중 y와 같은 값을 가지는 class의 index를 저장
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    # idxs ~ samples_per_class(7) 까지의 수 중 랜덤으로 하나 뽑기
    
    #print(idxs)


    # i : idxs의 index
    # idx : idxs의 값
    for i, idx in enumerate(idxs):
        #print(i, idx)
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(X_train[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

# Data preprocessing

np.reshape(변경할 배열, 변경될 차원)  
np.reshape에서 -1의 의미
- 남은 곳의 차원을 의미
- ex) reshape(4각정사각배열,(2,-1)) -> 행=2, 열=8

In [ ]:
# Subsample the data for more efficient code execution in this exercise
num_training = 5000
mask = list(range(num_training))
X_train = X_train[mask]
y_train = y_train[mask]
# training data를 5000개만 사용

num_test = 500
mask = list(range(num_test))
X_test = X_test[mask]
y_test = y_test[mask]
#test data를 500개만 사용
#print(X_train.shape)
#print(X_test.shape)
# Reshape the image data into rows
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
print(X_train.shape, X_test.shape)

In [ ]:
from cs231n.classifiers import KNearestNeighbor

# Create a kNN classifier instance. 
# Remember that training a kNN classifier is a noop: 
# the Classifier simply remembers the data and does no further processing 
classifier = KNearestNeighbor()
classifier.train(X_train, y_train)

We would now like to classify the test data with the kNN classifier. Recall that we can break down this process into two steps: 

1. First we must compute the distances between all test examples and all train examples. 
- 우리는 모든 test와 train에 대해 거리를 계산해야 한다.
2. Given these distances, for each test example we find the k nearest examples and have them vote for the label
- 우리는 이 거리를 통해, 각각의 test example에 대해 k nearest를 찾고, label을 위해 투표를 진행한다.

Lets begin with computing the distance matrix between all training and test examples. For example, if there are **Ntr** training examples and **Nte** test examples, this stage should result in a **Nte x Ntr** matrix where each element (i,j) is the distance between the i-th test and j-th train example.

**Note: For the three distance computations that we require you to implement in this notebook, you may not use the np.linalg.norm() function that numpy provides.**

First, open `cs231n/classifiers/k_nearest_neighbor.py` and implement the function `compute_distances_two_loops` that uses a (very inefficient) double loop over all pairs of (test, train) examples and computes the distance matrix one element at a time.
- k_nearest_neighbor.py를 통해 double loop를 구현하고 시험해보자

In [ ]:
# Open cs231n/classifiers/k_nearest_neighbor.py and implement
# compute_distances_two_loops.

# Test your implementation:
dists = classifier.compute_distances_two_loops(X_test)
print(dists.shape)

In [ ]:
# We can visualize the distance matrix: each row is a single test example and
# its distances to training examples
plt.imshow(dists, interpolation='none')
plt.show()

**Inline Question 1** 

Notice the structured patterns in the distance matrix, where some rows or columns are visibly brighter. (Note that with the default color scheme black indicates low distances while white indicates high distances.)
- 일부 행이나 열이 눈에 띄게 밝아지는 거리 행렬의 구조화된 패턴을 확인하십시오. (기본 색 구성표에서 검은색은 낮은 거리를 나타내고 흰색은 높은 거리를 나타냅니다.)

1. What in the data is the cause behind the distinctly bright rows?
  - 데이터에서 뚜렷하게 밝은 열의 원인은 무엇입니까?
2. What causes the columns?
  - 열의 원인은 무엇입니까?

$\color{blue}{\textit Your Answer:}$  
1. 테스트 데이터가 트레이닝 데이터와 서로 맞지 않아 생성된다.
   - i : test, j : train
2. 트레이닝 데이터가 테스트 데이터와 서로 맞지 않아 생성된다.




In [ ]:
# Now implement the function predict_labels and run the code below:
# We use k = 1 (which is Nearest Neighbor).
y_test_pred = classifier.predict_labels(dists, k=1)

# Compute and print the fraction of correctly predicted examples
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))

You should expect to see approximately `27%` accuracy. Now lets try out a larger `k`, say `k = 5`:

In [ ]:
y_test_pred = classifier.predict_labels(dists, k=5)
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))

You should expect to see a slightly better performance than with `k = 1`.

**Inline Question 2**

We can also use other distance metrics such as L1 distance.
For pixel values $p_{ij}^{(k)}$ at location $(i,j)$ of some image $I_k$,

the mean $\mu$ across all pixels over all images is $$\mu=\frac{1}{nhw}\sum_{k=1}^n\sum_{i=1}^{h}\sum_{j=1}^{w}p_{ij}^{(k)}$$
And the pixel-wise mean $\mu_{ij}$ across all images is 
$$\mu_{ij}=\frac{1}{n}\sum_{k=1}^np_{ij}^{(k)}.$$
The general standard deviation $\sigma$ and pixel-wise standard deviation $\sigma_{ij}$ is defined similarly.

Which of the following preprocessing steps will not change the performance of a Nearest Neighbor classifier that uses L1 distance? Select all that apply.
1. Subtracting the mean $\mu$ ($\tilde{p}_{ij}^{(k)}=p_{ij}^{(k)}-\mu$.)
2. Subtracting the per pixel mean $\mu_{ij}$  ($\tilde{p}_{ij}^{(k)}=p_{ij}^{(k)}-\mu_{ij}$.)
3. Subtracting the mean $\mu$ and dividing by the standard deviation $\sigma$.
4. Subtracting the pixel-wise mean $\mu_{ij}$ and dividing by the pixel-wise standard deviation $\sigma_{ij}$.
5. Rotating the coordinate axes of the data.

-> 문제 : 1,2,3,4,5 중 어떤 preprocessing을 하였을 때 L1 distance를 이용한 KNN이 영향을 받지 않는가.

1. 전체의 평균을 각 이미지의 픽셀에서 빼준다.
2. 각 픽셀의 평균을 각 이미지의 픽셀에서 빼준다.
3. 전체의 평균을 빼주고, 표준편차(standard deviation)으로 나눠준다.
4. 각 픽셀의 평균을 빼주고, 픽셀 별 표준편차로 나누어 준다.
5. data의 좌표를 회전시켜준다.

$\color{blue}{\textit Your Answer:}$  

answer : 1, 3


$\color{blue}{\textit Your Explanation:}$

explanation :  

1. 전체의 평균을 빼주므로 모든 픽셀들이 같은 방향으로 같은 양 만큼 움직이므로 L1 distance는 변하지 않는다.
2. 각 픽셀의 평균을 빼주는 것이므로 픽셀 별 평균 값이 달라 다른 방향, 다른 양 만큼 움직이므로 L1 distance에 영향을 줄 것이다.
3. -> 1번과 동일
4. -> 2번과 동일
5. 데이터의 축이 90도의 배수만큼 변할 시에는 L1 distance가 변하지 않지만, 이외의 경우 L1 distance가 변하게 되므로 영향을 끼친다. L2 distance의 경우 영향을 받지 않는다.

In [ ]:
# Now lets speed up distance matrix computation by using partial vectorization
# with one loop. Implement the function compute_distances_one_loop and run the
# code below:
dists_one = classifier.compute_distances_one_loop(X_test)

# To ensure that our vectorized implementation is correct, we make sure that it
# agrees with the naive implementation. There are many ways to decide whether
# two matrices are similar; one of the simplest is the Frobenius norm. In case
# you haven't seen it before, the Frobenius norm of two matrices is the square
# root of the squared sum of differences of all elements; in other words, reshape
# the matrices into vectors and compute the Euclidean distance between them.
difference = np.linalg.norm(dists - dists_one, ord='fro')
print('One loop difference was: %f' % (difference, ))
if difference < 0.001:
    print('Good! The distance matrices are the same')
else:
    print('Uh-oh! The distance matrices are different')

In [ ]:
# Now implement the fully vectorized version inside compute_distances_no_loops
# and run the code
dists_two = classifier.compute_distances_no_loops(X_test)

# check that the distance matrix agrees with the one we computed before:
difference = np.linalg.norm(dists - dists_two, ord='fro')
print('No loop difference was: %f' % (difference, ))
if difference < 0.001:
    print('Good! The distance matrices are the same')
else:
    print('Uh-oh! The distance matrices are different')

In [ ]:
# Let's compare how fast the implementations are
def time_function(f, *args):
    """
    Call a function f with args and return the time (in seconds) that it took to execute.
    """
    import time
    tic = time.time()
    f(*args)
    toc = time.time()
    return toc - tic

two_loop_time = time_function(classifier.compute_distances_two_loops, X_test)
print('Two loop version took %f seconds' % two_loop_time)

one_loop_time = time_function(classifier.compute_distances_one_loop, X_test)
print('One loop version took %f seconds' % one_loop_time)

no_loop_time = time_function(classifier.compute_distances_no_loops, X_test)
print('No loop version took %f seconds' % no_loop_time)

# You should see significantly faster performance with the fully vectorized implementation!

# NOTE: depending on what machine you're using, 
# you might not see a speedup when you go from two loops to one loop, 
# and might even see a slow-down.

### Cross-validation

We have implemented the k-Nearest Neighbor classifier but we set the value k = 5 arbitrarily. We will now determine the best value of this hyperparameter with cross-validation.

- k의 값을 임의로 설정했었다.
- k의 최적의 값을 cross-validation으로 구해보자

In [ ]:
num_folds = 5
# train data를 나눌 개수
k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]
# k 값 후보

X_train_folds = []
y_train_folds = []
################################################################################
# TODO:                                                                        #
# Split up the training data into folds. After splitting, X_train_folds and    #
# y_train_folds should each be lists of length num_folds, where                #
# y_train_folds[i] is the label vector for the points in X_train_folds[i].     #
# Hint: Look up the numpy array_split function.                                #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# folder의 개수에 따라 split
X_train_folds = np.array_split(X_train, num_folds)
y_train_folds = np.array_split(y_train, num_folds)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# A dictionary holding the accuracies for different values of k that we find
# when running cross-validation. After running cross-validation,
# k_to_accuracies[k] should be a list of length num_folds giving the different
# accuracy values that we found when using that value of k.
k_to_accuracies = {}

################################################################################
# TODO:                                                                        #
# Perform k-fold cross validation to find the best value of k. For each        #
# possible value of k, run the k-nearest-neighbor algorithm num_folds times,   #
# where in each case you use all but one of the folds as training data and the #
# last fold as a validation set. Store the accuracies for all fold and all     #
# values of k in the k_to_accuracies dictionary.                               #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

for k in k_choices:
    accuracy = []
    for i in range(num_folds):
        # i 번째만 validation set으로 빼주고 나머지는 합쳐서 train set으로 만들어주기
        # valid_set = X_train_folds[i]
        train_set = np.concatenate(X_train_folds[:i] + X_train_folds[i+1:])
        label_set = np.concatenate(y_train_folds[:i] + y_train_folds[i+1:])
        # classifier train -> train_set/label_set
        classifier.train(train_set, label_set)
        # k를 바꿔가며 prediction해보기
        valid_pred = classifier.predict(X_train_folds[i], k=k)
        # acc 구하기
        valid_acc = np.sum(valid_pred == y_train_folds[i])/ valid_pred.size
        accuracy.append(valid_acc)
    
    k_to_accuracies[k] = accuracy
# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# Print out the computed accuracies
for k in sorted(k_to_accuracies):
    for accuracy in k_to_accuracies[k]:
        print('k = %d, accuracy = %f' % (k, accuracy))

In [ ]:
# plot the raw observations
for k in k_choices:
    accuracies = k_to_accuracies[k]
    plt.scatter([k] * len(accuracies), accuracies)

# plot the trend line with error bars that correspond to standard deviation
accuracies_mean = np.array([np.mean(v) for k,v in sorted(k_to_accuracies.items())])
accuracies_std = np.array([np.std(v) for k,v in sorted(k_to_accuracies.items())])
plt.errorbar(k_choices, accuracies_mean, yerr=accuracies_std)
plt.title('Cross-validation on k')
plt.xlabel('k')
plt.ylabel('Cross-validation accuracy')
plt.show()

In [ ]:
# Based on the cross-validation results above, choose the best value for k,   
# retrain the classifier using all the training data, and test it on the test
# data. You should be able to get above 28% accuracy on the test data.
best_k = 1

classifier = KNearestNeighbor()
classifier.train(X_train, y_train)
y_test_pred = classifier.predict(X_test, k=best_k)

# Compute and display the accuracy
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))

**Inline Question 3**

Which of the following statements about $k$-Nearest Neighbor ($k$-NN) are true in a classification setting, and for all $k$? Select all that apply.
1. The decision boundary of the k-NN classifier is linear.
2. The training error of a 1-NN will always be lower than or equal to that of 5-NN.
3. The test error of a 1-NN will always be lower than that of a 5-NN.
4. The time needed to classify a test example with the k-NN classifier grows with the size of the training set.
5. None of the above.

$\color{blue}{\textit Your Answer:}$  

answer : 2, 4


$\color{blue}{\textit Your Explanation:}$  


1. KNN의 decision boundary = non-linear
2. training error란 training에 사용한 데이터를 test때 그대로 사용하는 것인데 1-NN의 경우 자기 자신을 찾아 distance가 0이 나올 것이지만, 5-NN의 경우 다른 것을 찾을 수도 있으므로 distance가 1-NN보다 항상 크다.
3. 이것은 확신할 수 없다.
4. training set의 크기가 커질 경우, KNN은 모든 train set과의 거리를 구하게 되므로 시간이 오래 걸리게 된다.
